In [1]:
import pandas as pd
from datetime import datetime
import os
import argparse

In [2]:
# 读取raw_data
output_path = './Results/size%d' % (1)
if not os.path.exists(output_path):
    # 目录不存在，进行创建操作
    os.makedirs(output_path)

startTime = datetime.now()
traj_undropped = pd.read_csv(output_path + '/raw_pos.txt',sep="\t")
# 删除日期未知的轨迹
traj_undropped.drop(traj_undropped[traj_undropped['Unnamed: 0'] == '*'].index, inplace=True)
traj_undropped.shape

(11142, 50)

In [3]:
# 删除采样点少于12个的轨迹
columns = traj_undropped.columns
traj_dropped = pd.DataFrame(columns=columns)
for i in range(len(traj_undropped)):
    if '*' in traj_undropped.iloc[i].value_counts():
        count = traj_undropped.iloc[i].value_counts()['*']
    else:
        count = 0
    if count >= 36:
        traj_dropped = traj_dropped.append(traj_undropped.iloc[i])

# traj_undropped.index = list(traj_undropped.iloc[:,0])
print(traj_dropped.shape)


(9252, 50)


In [4]:
# 删除轨迹少于5条的用户
traj_drop_uid = pd.DataFrame(columns=columns)
# 每个用户的天数
traj_dropped.index = list(traj_dropped.iloc[:,0])
date_counts = traj_dropped.index.value_counts()

for uid in date_counts.index:
    if date_counts[uid] > 5:
        traj_drop_uid = traj_drop_uid.append(traj_dropped.loc[uid])
print(traj_drop_uid.shape)

(9144, 50)


In [5]:
endTime = datetime.now()
print("Dropping time:",str(endTime-startTime)[0:7])

Dropping time: 0:02:34


In [6]:
# 删除小数点
traj_drop_uid.replace('*', -1, inplace=True)
traj_drop_uid.iloc[:, 2:] = traj_drop_uid.iloc[:, 2:].astype(float)
traj_drop_uid.iloc[:, 2:] = traj_drop_uid.iloc[:, 2:].astype(int)
traj_drop_uid.replace(-1, '*', inplace=True)
print(set(traj_drop_uid['Unnamed: 0']))
print(len(set(traj_drop_uid['Unnamed: 0'])))

{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 22, 23, 24, 25, 26, 28, 29, 30, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 50, 51, 52, 53, 55, 56, 57, 58, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 73, 74, 75, 76, 78, 79, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 108, 110, 111, 112, 113, 114, 115, 117, 119, 122, 125, 126, 128, 129, 130, 131, 134, 135, 136, 138, 139, 140, 141, 142, 144, 146, 147, 150, 153, 154, 155, 157, 158, 159, 161, 162, 163, 164, 165, 167, 168, 169, 172, 173, 174, 176, 179, 181}
143


In [7]:
#给用户重新编号
keys = set(traj_drop_uid.index)
values = [i for i in range(1, len(keys) + 1)]
tmp_dict = dict(zip(keys, values))

traj_drop_uid['Unnamed: 0'] = traj_drop_uid['Unnamed: 0'].replace(tmp_dict)
traj_drop_uid.sort_values(by="Unnamed: 0" , inplace=True, ascending=True)
print(traj_drop_uid.shape)
print(set(traj_drop_uid['Unnamed: 0']))
print(len(set(traj_drop_uid['Unnamed: 0'])))


(9144, 50)
{1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143}
143


In [8]:
print("generate dropped_pos.txt...")
traj_drop_uid.to_csv(output_path + '/dropped_pos.txt',sep='\t',index=False,header=1)
print("generatation completed.")

generate dropped_pos.txt...
generatation completed.
